In [174]:
import pandas as pd
from carps.analysis.gather_data import normalize_logs, get_interpolated_performance_df
from numpy import trapz
import importlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px 
import itertools
import pandas as pd

import carps.analysis.performance_over_time
from carps.analysis.generate_report import *


importlib.reload(carps)
importlib.reload(carps.analysis)
importlib.reload(carps.analysis.performance_over_time)
from carps.analysis.performance_over_time import plot_performance_over_time, setup_seaborn, savefig, get_color_palette, get_order_by_mean


# Old

In [58]:
tasks = {}
tasks['Pymoo'] = [
'multi-objective/50/dev/Pymoo/MO/unconstraint/kursawe',
'multi-objective/50/dev/Pymoo/MO/unconstraint/zdt1',
'multi-objective/50/dev/Pymoo/MO/unconstraint/zdt2',
'multi-objective/50/dev/Pymoo/MO/unconstraint/zdt6']
tasks['HPOBench'] = ['multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/53',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/9952', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/9977', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/10101', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/146212', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/146606',  
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/146818',  
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/146821', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/146822', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/167119', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/lr/167120', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/nn/3917', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/nn/10101',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/nn/146821', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/rf/3',  
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/rf/9952',  
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/rf/146212',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/rf/167119',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/rf/167120',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/rf/168911',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/3', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/31', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/53', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/3917',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/9952',  
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/9977',  
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/14965', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/146822',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/svm/168911',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/xgboost/3917',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/xgboost/7592', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/xgboost/9952', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/xgboost/146606',
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/xgboost/146822', 
       'multi-objective/50/dev/hpobench/multiobjective/tabular/ml/xgboost/167120']
tasks['YAHPO'] = ['multi-objective/50/dev/yahpo/mo/iaml_glmnet/1489/None',
      'multi-objective/50/dev/yahpo/mo/iaml_ranger/1489/None', 
      'multi-objective/50/dev/yahpo/mo/rbv2_ranger/375/None', 
      'multi-objective/50/dev/yahpo/mo/rbv2_xgboost/28/None', 
      'multi-objective/50/dev/yahpo/mo/rbv2_xgboost/182/None']

In [53]:

def expand_optimizer_id(df: pd.DataFrame, optimizer_id_col: str = 'optimizer_id') -> pd.DataFrame:
    def parse_optimizer_id(opt_id: str) -> dict:
        info = {
            'hpi_method': None,
            'adjust_cs': 'False',
            'cs_method': 'False',
            'pc_method': 'False',
            'thresh': None,
            'adjust_method': None
        }
        parts = opt_id.split('_')
        info['hpi_method'] = parts[0]

        
        if 'adjust_cs_' in opt_id:
            info['adjust_cs'] = 'distribution'
            info['cs_method'] = parts[parts.index('cs')+1]
            info['adjust_method'] = 'adjust_cs_only'
        if 'constant' in opt_id:
            info['adjust_cs'] = 'constant'
        if 'cs_proba_hpi' in opt_id:
            info['adjust_cs'] = 'cs_proba_hpi'
        if 'set_to' in opt_id:
            info['pc_method'] = parts[parts.index('to')+1]
            if info['adjust_method'] == 'adjust_cs_only':
                info['adjust_method'] = 'both'
            else:
                info['adjust_method'] = 'adjust_pc_only'
        if 'down' in opt_id:
            info['thresh'] = 'down'
        elif 'up' in opt_id:
            info['thresh'] = 'up'
        else: 
            info['thresh'] = parts[parts.index('thresh')+1]
        
        return info

    # Apply the parser to each row
    parsed = df[optimizer_id_col].apply(parse_optimizer_id)
    parsed_df = pd.DataFrame(parsed.tolist())

    # Merge with the original DataFrame
    return pd.concat([df, parsed_df], axis=1)

def check_if_complete_for_max_trials(in_df):
    max_trials = in_df['n_trials_norm'].max()
    filtered_df = in_df[in_df['n_trials_norm'] == max_trials]
    miss_dict = {}
    seeds = filtered_df['seed'].unique()
    expected_combinations = set(itertools.product(tasks[in_df['benchmark_id'].iloc[0]], [max_trials], seeds))
    incomplete = False
    for optimizer_id, group in filtered_df.groupby('optimizer_id'):
        actual_combinations = set(group[['task_id', 'n_trials_norm', 'seed']].itertuples(index=False, name=None))
        missing = expected_combinations - actual_combinations
        if missing:
            print(f"Optimizer {optimizer_id} is not complete for max n_trials_norm = {max_trials}: {len(missing)} combinations missing.")
            miss_dict[optimizer_id] = missing
            incomplete = True
    if not incomplete:
        print('All data complete!')
        return None
    return miss_dict

def read_data(path):
    df = pd.read_parquet(path)
    df = normalize_logs(df)
    df['n_trials_norm'] = df.groupby('experiment_id')['time'].rank(method='dense', pct=True).round(2)
    if 'set' not in df.columns:
        df["set"] = '0'
    df_random = df[df['optimizer_id'].str.startswith('random_')]
    df = df[~df['optimizer_id'].str.startswith('random_')]
    # check_if_complete_for_max_trials(df)
    return df, df_random

def get_perf(df):
    perf = get_interpolated_performance_df(df, n_points=50)
    missing = check_if_complete_for_max_trials(df)
    perf = expand_optimizer_id(perf)
    perf = perf[perf['adjust_method'].notna()]
    return perf, missing

In [153]:
fn = "/home/daphne/Code/hpi_parego/experimenter/smac_rf/processed_logs.parquet"
base = pd.read_parquet(fn)
base = normalize_logs(base)
base['n_trials_norm'] = base.groupby('experiment_id')['time'].rank(method='dense', pct=True).round(2)
if 'set' not in base.columns:
    base["set"] = '0'
# base = base[~base['task_id'].str.contains('ManyO')]
base['optimizer_id'] = 'ParEGO'
base_test = base[base['subset_id']=='test']
base_train = base[base['subset_id']=='dev']
base_train = get_interpolated_performance_df(base_train, n_points=50)
missing = check_if_complete_for_max_trials(base_train)
base_test = get_interpolated_performance_df(base_test, n_points=50)

[11:10:49] INFO     Start normalization...                                                       ]8;id=872248;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=779619;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#616\616]8;;\

           INFO     Normalize n_trials...                                                        ]8;id=724489;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=171516;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#617\617]8;;\

           INFO     Normalize cost...                                                            ]8;id=510158;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=223311;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#619\619]8;;\

           INFO     Calc normalized incumbent cost...                                            ]8;id=494688;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=130571;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#628\628]8;;\

           INFO     Normalize time...                                                            ]8;id=291220;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=961269;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#648\648]8;;\

           INFO     Done.                                                                        ]8;id=750276;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=874192;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#651\651]8;;\

           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=869128;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=393294;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

All data complete!


           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=500542;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=394479;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

In [157]:
path = "/home/daphne/Code/hpi_parego/experimenter/best_test/processed_logs.parquet"
best_test, best_test_rnd = read_data(path)
best_test = best_test[~best_test['optimizer_id'].str.startswith('hypershap')]
# best_test = best_test[~best_test['task_id'].str.contains('ManyO')]
best_test, _ = get_perf(best_test)
best_test_rnd, _ = get_perf(best_test_rnd)

[11:12:14] INFO     Start normalization...                                                       ]8;id=544939;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=961515;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#616\616]8;;\

           INFO     Normalize n_trials...                                                        ]8;id=402918;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=542128;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#617\617]8;;\

           INFO     Normalize cost...                                                            ]8;id=573636;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=611362;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#619\619]8;;\

           INFO     Calc normalized incumbent cost...                                            ]8;id=162731;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=536345;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#628\628]8;;\

           INFO     Normalize time...                                                            ]8;id=221103;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=369826;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#648\648]8;;\

           INFO     Done.                                                                        ]8;id=472057;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=103946;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#651\651]8;;\

           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=708164;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=30346;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

Optimizer fanova_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_thresh_0.75 is not complete for max n_trials_norm = 1.0: 25 combinations missing.
Optimizer fanova_adjust_cs_incumbent_cs_proba_hpi_thresh_0.5 is not complete for max n_trials_norm = 1.0: 25 combinations missing.


           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=645079;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=620125;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

Optimizer random_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down is not complete for max n_trials_norm = 1.0: 20 combinations missing.
Optimizer random_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_thresh_0.75 is not complete for max n_trials_norm = 1.0: 20 combinations missing.
Optimizer random_adjust_cs_incumbent_cs_proba_hpi_thresh_0.5 is not complete for max n_trials_norm = 1.0: 20 combinations missing.
Optimizer random_adjust_cs_random_adjust_prev_cfgs_set_to_random_thresh_0.75 is not complete for max n_trials_norm = 1.0: 20 combinations missing.


In [63]:
path = "/home/daphne/Code/hpi_parego/experimenter/yahpo/processed_logs.parquet"
yahpo_df, yahpo_rnd = read_data(path)
yahpo_df = yahpo_df[~yahpo_df['optimizer_id'].str.startswith('hypershap')]
yahpo_perf, missing = get_perf(yahpo_df)
yahpo_rnd, _ = get_perf(yahpo_rnd)

[10:17:27] INFO     Start normalization...                                                       ]8;id=569875;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=751225;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#616\616]8;;\

           INFO     Normalize n_trials...                                                        ]8;id=586676;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=155679;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#617\617]8;;\

           INFO     Normalize cost...                                                            ]8;id=12318;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=984020;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#619\619]8;;\

           INFO     Calc normalized incumbent cost...                                            ]8;id=779149;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=290199;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#628\628]8;;\

           INFO     Normalize time...                                                            ]8;id=512061;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=279155;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#648\648]8;;\

           INFO     Done.                                                                        ]8;id=251531;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=15549;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#651\651]8;;\

           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=683244;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=635539;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

All data complete!


[10:17:30] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=270168;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=26573;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

All data complete!


In [64]:
path = "/home/daphne/Code/hpi_parego/experimenter/hpobench_part_1/processed_logs.parquet"
hpo1_df, hpo1_rnd = read_data(path)
path = "/home/daphne/Code/hpi_parego/experimenter/hpobench_part_2/processed_logs.parquet"
hpo2_df, hpo2_rnd = read_data(path)
hpo_df = pd.concat([hpo1_df, hpo2_df])
hpo_df = hpo_df[~hpo_df['optimizer_id'].str.startswith('hypershap')]
hpo_rnd = pd.concat([hpo1_rnd, hpo2_rnd])
hpo_df = hpo_df.sort_values('trial_value__endtime').drop_duplicates(subset=['task_id', 'optimizer_id', 'seed', 'n_trials_norm'], keep='last')
hpo_rnd = hpo_rnd.sort_values('trial_value__endtime').drop_duplicates(subset=['task_id', 'optimizer_id', 'seed', 'n_trials_norm'], keep='last')
hpo_perf, missing = get_perf(hpo_df)
hpo_rnd, _ = get_perf(hpo_rnd)

[10:17:32] INFO     Start normalization...                                                       ]8;id=904773;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=967203;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#616\616]8;;\

           INFO     Normalize n_trials...                                                        ]8;id=506730;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=993355;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#617\617]8;;\

           INFO     Normalize cost...                                                            ]8;id=187242;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=338480;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#619\619]8;;\

           INFO     Calc normalized incumbent cost...                                            ]8;id=479801;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=63538;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#628\628]8;;\

[10:17:33] INFO     Normalize time...                                                            ]8;id=344867;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=709403;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#648\648]8;;\

...

[10:17:34] INFO     Done.                                                                        ]8;id=522559;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=500185;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#651\651]8;;\

           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=707133;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=733249;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

All data complete!


[10:17:50] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=777681;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=190728;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

Optimizer random__adjust_prev_cfgs_set_to_default_dynamic_decay_down is not complete for max n_trials_norm = 1.0: 73 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_default_dynamic_decay_up is not complete for max n_trials_norm = 1.0: 69 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_default_thresh_0.5 is not complete for max n_trials_norm = 1.0: 71 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_default_thresh_0.75 is not complete for max n_trials_norm = 1.0: 61 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down is not complete for max n_trials_norm = 1.0: 78 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_incumbent_dynamic_decay_up is not complete for max n_trials_norm = 1.0: 82 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_incumbent_thresh_0.5 is not complete for max n_trials_norm = 1.0: 68 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_incumb

In [65]:
path = "/home/daphne/Code/hpi_parego/experimenter/pymoo_part_1/processed_logs.parquet"
pymoo1_df, pymoo1_rnd = read_data(path)
path = "/home/daphne/Code/hpi_parego/experimenter/pymoo_part_2/processed_logs.parquet"
pymoo2_df, pymoo2_rnd = read_data(path)
pymoo_df = pd.concat([pymoo1_df, pymoo2_df])
pymoo_df = pymoo_df[~pymoo_df['optimizer_id'].str.startswith('hypershap')]
pymoo_rnd = pd.concat([pymoo1_rnd, pymoo2_rnd])
pymoo_df = pymoo_df.sort_values('trial_value__endtime').drop_duplicates(subset=['task_id', 'optimizer_id', 'seed', 'n_trials_norm'], keep='last')
pymoo_rnd = pymoo_rnd.sort_values('trial_value__endtime').drop_duplicates(subset=['task_id', 'optimizer_id', 'seed', 'n_trials_norm'], keep='last')
pymoo_perf, missing = get_perf(pymoo_df)
pymoo_rnd, _ = get_perf(pymoo_rnd)

[10:17:57] INFO     Start normalization...                                                       ]8;id=83241;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=916485;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#616\616]8;;\

           INFO     Normalize n_trials...                                                        ]8;id=121968;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=53067;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#617\617]8;;\

           INFO     Normalize cost...                                                            ]8;id=132210;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=73725;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#619\619]8;;\

           INFO     Calc normalized incumbent cost...                                            ]8;id=426688;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=513068;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#628\628]8;;\

[10:17:58] INFO     Normalize time...                                                            ]8;id=564443;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=949589;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#648\648]8;;\

...

           INFO     Done.                                                                        ]8;id=181047;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=550789;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#651\651]8;;\

           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=803837;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=711481;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

All data complete!


[10:18:00] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=404313;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=362972;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

Optimizer random__adjust_prev_cfgs_set_to_default_dynamic_decay_down is not complete for max n_trials_norm = 1.0: 1 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_default_thresh_0.5 is not complete for max n_trials_norm = 1.0: 5 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_default_thresh_0.75 is not complete for max n_trials_norm = 1.0: 4 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down is not complete for max n_trials_norm = 1.0: 2 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_incumbent_thresh_0.5 is not complete for max n_trials_norm = 1.0: 4 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_incumbent_thresh_0.75 is not complete for max n_trials_norm = 1.0: 3 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_random_dynamic_decay_down is not complete for max n_trials_norm = 1.0: 3 combinations missing.
Optimizer random__adjust_prev_cfgs_set_to_random_dynamic_de

In [67]:
perf_all = pd.concat([yahpo_perf, hpo_perf, pymoo_perf])
check_if_complete_for_max_trials(perf_all)
rnd_all = pd.concat([yahpo_rnd, hpo_rnd, pymoo_rnd])

All data complete!


In [68]:
def plot_one_thing(thing, df_plot=None):
    if df_plot is None:
        df_plot = perf_all.copy(deep=True)
        df_plot['optimizer_id'] = df_plot[thing]
        # df_plot = pd.concat([df_plot, base], ignore_index=True)
        plot_performance_per_task(df_plot, output_dir=f"figure_dir/ablation/{thing}", replot=True)
        if len(df_plot['optimizer_id'].unique()) > 2:
            plot_critical_difference(df_plot, output_dir=f"figure_dir/ablation/{thing}", replot=True)
            plot_finalperfbarplot(df_plot, output_dir=f"figure_dir/ablation/{thing}", replot=True)
            plot_ranks_over_time(df_plot, output_dir=f"figure_dir/ablation/{thing}", replot=True)

    # Average over tasks
    avg_task = df_plot.groupby(["optimizer_id", "seed", "n_trials_norm"]).agg(
        mean_perf=("trial_value__cost_inc_norm", "mean")
    ).reset_index()

    # Average over seeds
    avg_seed = avg_task.groupby(["optimizer_id", "n_trials_norm"]).agg(
        mean=("mean_perf", "mean"),
        sem=("mean_perf", "sem")
    ).reset_index()

    auc_per_seed = (
        df_plot.groupby(["optimizer_id", "seed"])
        .apply(lambda group: trapz(group.sort_values("n_trials_norm")["trial_value__cost_inc_norm"],
                                   group.sort_values("n_trials_norm")["n_trials_norm"]))
        .reset_index(name="auc")
    )

    # Compute mean and sem AUC per optimizer
    auc_values = auc_per_seed.groupby("optimizer_id").agg(
        mean=('auc', 'mean'),
        sem=('auc', 'sem')
    ).sort_values("mean", ascending=False).reset_index()


    fig = make_subplots(
        rows=1, cols=2,
        # subplot_titles=("Performance over Time (Normalized)", "AUC per Optimizer"),
        column_widths=[0.6, 0.4],
        horizontal_spacing=0.15
    )
    colors = px.colors.qualitative.Set1
    # Line plot with shaded error (left)
    print(avg_seed["optimizer_id"].unique())
    for i, optimizer in enumerate(avg_seed["optimizer_id"].unique()):
        df_opt = avg_seed[avg_seed["optimizer_id"] == optimizer]
        
        line_color = colors[i % len(colors)]

        # Mean line
        fig.add_trace(go.Scatter(
            x=df_opt["n_trials_norm"],
            y=df_opt["mean"],
            mode="lines",
            name=optimizer,
            line=dict(width=2, color=line_color),
        ), row=1, col=1)

        # Shaded error (sem)
        fig.add_trace(go.Scatter(
            x=pd.concat([df_opt["n_trials_norm"], df_opt["n_trials_norm"][::-1]]),
            y=pd.concat([
                df_opt["mean"] - df_opt["sem"],
                (df_opt["mean"] + df_opt["sem"])[::-1]
            ]),
            fill='toself',
            fillcolor=line_color.replace(')', ',0.1)').replace('rgb', 'rgba'),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=False
        ), row=1, col=1)

    # Bar chart with error bars (right)
    fig.add_trace(go.Bar(
        x=auc_values["mean"],
        y=auc_values["optimizer_id"],
        orientation="h",
        error_x=dict(type='data', array=auc_values["sem"]),
        marker=dict(color="skyblue", line=dict(color="black", width=1)),
        showlegend=False
    ), row=1, col=2)

    # -------------------------
    # 3. Layout & Font
    # -------------------------

    fig.update_layout(
        height=400,
        width=1200,
        template="plotly_white",
        font=dict(size=16),
        legend=dict(title="Optimizers", font=dict(size=16)),
        margin=dict(t=60, l=50, r=50, b=50)
    )

    fig.update_xaxes(title_text="Normalized Trials", row=1, col=1)
    fig.update_yaxes(title_text="Normalized Inc-Performance", row=1, col=1)
    fig.update_xaxes(title_text="AUC", row=1, col=2)
    fig.update_yaxes(title_text="Optimizer", row=1, col=2)

    fig.show()


In [69]:
# Which HPI method
plot_one_thing('hpi_method')

['fanova']


/tmp/ipykernel_4800/1961929167.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [70]:
# Which thresholding?
plot_one_thing('thresh')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[10:20:03] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=594240;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=286116;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['0.5' '0.75' 'down' 'up']


/tmp/ipykernel_4800/1961929167.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [71]:
plot_one_thing('adjust_cs')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[10:20:36] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=125005;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=654237;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['False' 'constant' 'cs_proba_hpi' 'distribution']


/tmp/ipykernel_4800/1961929167.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [72]:
plot_one_thing('cs_method')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[10:20:55] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=548492;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=107804;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['False' 'default' 'incumbent' 'random']


/tmp/ipykernel_4800/1961929167.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [73]:
plot_one_thing('pc_method')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[10:21:11] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=453705;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=213087;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['False' 'default' 'incumbent' 'random']


/tmp/ipykernel_4800/1961929167.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [74]:
plot_one_thing('adjust_method')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[10:21:28] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=564222;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=125380;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!



['adjust_cs_only' 'adjust_pc_only' 'both']


/tmp/ipykernel_4800/1961929167.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [215]:
perf_all['base'] = 'HPI'
perf_all.loc[perf_all['hpi_method'] == 'random', 'base'] = 'Random'

In [216]:
plot_one_thing('base')

['HPI']


/tmp/ipykernel_38293/1961929167.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [169]:
#test
perf_all = pd.concat([best_test, base_test], ignore_index=True)

In [170]:
auc_per_seed = (
        perf_all.groupby(["optimizer_id", "seed"])
        .apply(lambda group: trapz(group.sort_values("n_trials_norm")["trial_value__cost_inc_norm"],
                                   group.sort_values("n_trials_norm")["n_trials_norm"]))
        .reset_index(name="auc")
    )

# Compute mean and sem AUC per optimizer
auc_values = auc_per_seed.groupby("optimizer_id").agg(
    auc_mean=('auc', 'mean'),
    auc_sem=('auc', 'sem')
).sort_values("auc_mean", ascending=False).reset_index()


/tmp/ipykernel_4800/468351606.py:3: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [171]:
merge = pd.merge(auc_values, perf_all, on='optimizer_id', how='left')
merge = merge.drop_duplicates(subset=['optimizer_id'])
merge = merge[['optimizer_id', 'hpi_method', 'adjust_cs', 'cs_method', 'pc_method', 'thresh', 'adjust_method', 'auc_mean', 'auc_sem']]
final_perf = perf_all[perf_all['n_trials_norm'] == 1].groupby(['optimizer_id','seed']).agg({'trial_value__cost_inc_norm':'mean'}).groupby('optimizer_id').agg(
    final_perf_mean=('trial_value__cost_inc_norm', 'mean'),
    final_perf_sem=('trial_value__cost_inc_norm', 'sem')
).sort_values("final_perf_mean", ascending=False).reset_index()

merge = pd.merge(final_perf, merge, on='optimizer_id', how='right')
merge = merge[['optimizer_id','hpi_method', 'adjust_cs', 'cs_method', 'pc_method', 'thresh', 'adjust_method', 'auc_mean','auc_sem', 'final_perf_mean', 'final_perf_sem']].sort_values(by='final_perf_mean', ascending=True)
merge = merge.apply(lambda col: col.round(3) if col.name in ['auc_mean', 'auc_sem', 'final_perf_mean', 'final_perf_sem'] else col)
merge[['hpi_method', 'adjust_cs', 'cs_method', 'pc_method', 'thresh', 'adjust_method', 'auc_mean','auc_sem', 'final_perf_mean', 'final_perf_sem']].head(20).to_latex()

'\\begin{tabular}{lllllllrrrr}\n\\toprule\n & hpi_method & adjust_cs & cs_method & pc_method & thresh & adjust_method & auc_mean & auc_sem & final_perf_mean & final_perf_sem \\\\\n\\midrule\n1 & NaN & NaN & NaN & NaN & NaN & NaN & 0.237000 & 0.023000 & 0.036000 & 0.003000 \\\\\n2 & fanova & constant & default & incumbent & 0.75 & both & 0.227000 & 0.016000 & 0.043000 & 0.003000 \\\\\n0 & fanova & cs_proba_hpi & incumbent & False & 0.5 & adjust_cs_only & 0.241000 & 0.018000 & 0.048000 & 0.005000 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [149]:
merge.sort_values(by='auc_mean', ascending=True)

,optimizer_id,hpi_method,adjust_cs,cs_method,pc_method,thresh,adjust_method,auc_mean,auc_sem,final_perf_mean,final_perf_sem
2,fanova_adjust_cs_default_constant_adjust_prev_...,fanova,constant,default,incumbent,0.75,both,0.232,0.024,0.032,0.004
1,ParEGO,NaN,NaN,NaN,NaN,NaN,NaN,0.238,0.027,0.028,0.003
0,fanova_adjust_cs_incumbent_cs_proba_hpi_thresh...,fanova,cs_proba_hpi,incumbent,False,0.5,adjust_cs_only,0.243,0.026,0.034,0.005


In [172]:
# plot best fanova vs best hypershap vs ParEGO
best_final = merge[merge['final_perf_mean'] == merge['final_perf_mean'].min()]['optimizer_id'].values[0]
fanova_final = perf_all[perf_all['optimizer_id']==best_final]
fanova_final['optimizer_id'] = 'fANOVA-ParEGO-final'
best_auc = merge[merge['auc_mean'] == merge['auc_mean'].min()]['optimizer_id'].values[0]
fanova_auc = perf_all[perf_all['optimizer_id']==best_auc]
fanova_auc['optimizer_id'] = 'fANOVA-ParEGO-auc'
# hypershap_final = perf_all[perf_all['optimizer_id']=='hypershap_adjust_cs_random_adjust_prev_cfgs_set_to_random_thresh_0.75']
# hypershap_final['optimizer_id'] = 'HyperSHAP-ParEGO-final'
# hypershap_auc = perf_all[perf_all['optimizer_id']=='hypershap_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down']
# hypershap_auc['optimizer_id'] = 'HyperSHAP-ParEGO'
df_plot = pd.concat([base_test, fanova_auc, fanova_final], ignore_index=True)

plot_one_thing('optimizer_id', df_plot=df_plot)

['ParEGO' 'fANOVA-ParEGO-auc' 'fANOVA-ParEGO-final']


/tmp/ipykernel_4800/1280240574.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_4800/1280240574.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_4800/1961929167.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after g

In [163]:
df_plot = df_plot[df_plot['task_id'].str.contains('Pymoo')]

In [173]:
plot_performance_over_time(df_plot, y='trial_value__cost_inc_norm')
plot_critical_difference(df_plot, output_dir='figure_dir', replot=True)
plot_performance_per_task(df_plot, output_dir='figure_dir', replot=True)
plot_finalperfbarplot(df_plot, output_dir='figure_dir', replot=True)
plot_ranks_over_time(df_plot, output_dir='figure_dir', replot=True)


/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[11:23:29] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=379600;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=433534;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#725\725]8;;\
                    budget.                                                                                        

/home/daphne/miniforge3/envs/carps/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9064: RuntimeWarning:

invalid value encountered in scalar divide

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysi

[{'task_type': 'multi-objective',
  'set': '0',
  'task_id': None,
  'filename': 'figure_dir/rank_multi-objective_0',
  'plot_type': 'rank_over_time',
  'plot_type_pretty': 'Rank over Time',
  'explanation': 'The rank of each optimizer over time compares which optimizer performs better, the lower the rank the better. For each optimizer and task, the performance is averaged over seeds to obtain an estimate of the performance. The rank is then calculated per step and task with the same approach as for the critical difference diagram.'}]

# Random

In [186]:
fanova = perf_all[perf_all['optimizer_id']=='fanova_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_thresh_0.75']
fanova_rnd = rnd_all[rnd_all['optimizer_id']=='random_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_thresh_0.75']
fanova = pd.merge(fanova, fanova_rnd, on=['task_id', 'seed', 'n_trials_norm'], suffixes=('_final', '_rnd'))
fanova['trial_value__cost_inc_norm_diff'] = fanova['trial_value__cost_inc_norm_rnd'] - fanova['trial_value__cost_inc_norm_final']
fanova['optimizer_id'] = 'fANOVA-ParEGO'


hypershap = perf_all[perf_all['optimizer_id']=='hypershap_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down']
hypershap_rnd = rnd_all[rnd_all['optimizer_id']=='random_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down']
hypershap = pd.merge(hypershap, hypershap_rnd, on=['task_id', 'seed', 'n_trials_norm'], suffixes=('_final', '_rnd'))
hypershap['trial_value__cost_inc_norm_diff'] = hypershap['trial_value__cost_inc_norm_rnd'] - hypershap['trial_value__cost_inc_norm_final']
hypershap['optimizer_id'] = 'HyperSHAP-ParEGO'

df_plot = pd.concat([hypershap, fanova], ignore_index=True)

# Average over tasks
avg_task = df_plot.groupby(["optimizer_id", "seed", "n_trials_norm"]).agg(
    mean_perf=("trial_value__cost_inc_norm_diff", "mean")
).reset_index()

# Average over seeds
avg_seed = avg_task.groupby(["optimizer_id", "n_trials_norm"]).agg(
    mean=("mean_perf", "mean"),
    sem=("mean_perf", "sem")
).reset_index()

auc_per_seed = (
    df_plot.groupby(["optimizer_id", "seed"])
    .apply(lambda group: trapz(group.sort_values("n_trials_norm")["trial_value__cost_inc_norm_diff"],
                                group.sort_values("n_trials_norm")["n_trials_norm"]))
    .reset_index(name="auc")
)

colors = px.colors.qualitative.Set1
fig = go.Figure()
for i, optimizer in enumerate(avg_seed["optimizer_id"].unique()):
    df_opt = avg_seed[avg_seed["optimizer_id"] == optimizer]
    line_color = colors[i % len(colors)]

    # Mean line
    fig.add_trace(go.Scatter(
        x=df_opt["n_trials_norm"],
        y=df_opt["mean"],
        mode="lines",
        name=optimizer,
        line=dict(width=2, color=line_color),
    ))

    # Shaded error (sem)
    fig.add_trace(go.Scatter(
        x=pd.concat([df_opt["n_trials_norm"], df_opt["n_trials_norm"][::-1]]),
        y=pd.concat([
            df_opt["mean"] - df_opt["sem"],
            (df_opt["mean"] + df_opt["sem"])[::-1]
        ]),
        fill='toself',
        fillcolor=line_color.replace(')', ',0.1)').replace('rgb', 'rgba'),
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))


fig.update_layout(
    height=400,
    width=1100,
    template="plotly_white",
    font=dict(size=16),
    legend=dict(title="Optimizers", font=dict(size=16)),
    margin=dict(t=60, l=50, r=50, b=50)
)

fig.update_xaxes(title_text="Normalized Trials")
fig.update_yaxes(title_text="Difference in Performance HPI and Random")
fig.show()

/tmp/ipykernel_38293/129696559.py:29: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,task_id,seed,n_trials_norm,diff_trial_value__cost_inc_norm
0,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.00,0.000000
1,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.02,0.000000
2,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.04,0.000000
3,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.06,0.000000
4,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.08,0.000000
...,...,...,...,...
6013,multi-objective/50/dev/hpobench/multiobjective...,4,0.92,-0.005848
6014,multi-objective/50/dev/hpobench/multiobjective...,4,0.94,-0.005809
6015,multi-objective/50/dev/hpobench/multiobjective...,4,0.96,-0.005769
6016,multi-objective/50/dev/hpobench/multiobjective...,4,0.98,-0.005730


In [ ]:
def plot_diff_rnd(opt_id):
    hpi = perf_all[perf_all['optimizer_id']==opt_id]
    rnd = rnd_all[perf_all['optimizer_id']==opt_id]

    df_plot = perf_all.copy(deep=True)
    df_plot['optimizer_id'] = df_plot[thing]
    df_plot = pd.concat([df_plot, base], ignore_index=True)

    # Average over tasks
    avg_task = df_plot.groupby(["optimizer_id", "seed", "n_trials_norm"]).agg(
        mean_perf=("trial_value__cost_inc_norm", "mean")
    ).reset_index()

    # Average over seeds
    avg_seed = avg_task.groupby(["optimizer_id", "n_trials_norm"]).agg(
        mean=("mean_perf", "mean"),
        sem=("mean_perf", "sem")
    ).reset_index()

    auc_per_seed = (
        df_plot.groupby(["optimizer_id", "seed"])
        .apply(lambda group: trapz(group.sort_values("n_trials_norm")["trial_value__cost_inc_norm"],
                                   group.sort_values("n_trials_norm")["n_trials_norm"]))
        .reset_index(name="auc")
    )

    # Compute mean and sem AUC per optimizer
    auc_values = auc_per_seed.groupby("optimizer_id").agg(
        mean=('auc', 'mean'),
        sem=('auc', 'sem')
    ).sort_values("mean", ascending=False).reset_index()


    fig = make_subplots(
        rows=1, cols=2,
        # subplot_titles=("Performance over Time (Normalized)", "AUC per Optimizer"),
        column_widths=[0.6, 0.4],
        horizontal_spacing=0.15
    )
    colors = px.colors.qualitative.Set1
    # Line plot with shaded error (left)
    print(avg_seed["optimizer_id"].unique())
    for i, optimizer in enumerate(avg_seed["optimizer_id"].unique()):
        df_opt = avg_seed[avg_seed["optimizer_id"] == optimizer]
        
        line_color = colors[i % len(colors)]

        # Mean line
        fig.add_trace(go.Scatter(
            x=df_opt["n_trials_norm"],
            y=df_opt["mean"],
            mode="lines",
            name=optimizer,
            line=dict(width=2, color=line_color),
        ), row=1, col=1)

        # Shaded error (sem)
        fig.add_trace(go.Scatter(
            x=pd.concat([df_opt["n_trials_norm"], df_opt["n_trials_norm"][::-1]]),
            y=pd.concat([
                df_opt["mean"] - df_opt["sem"],
                (df_opt["mean"] + df_opt["sem"])[::-1]
            ]),
            fill='toself',
            fillcolor=line_color.replace(')', ',0.1)').replace('rgb', 'rgba'),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=False
        ), row=1, col=1)

    # Bar chart with error bars (right)
    fig.add_trace(go.Bar(
        x=auc_values["mean"],
        y=auc_values["optimizer_id"],
        orientation="h",
        error_x=dict(type='data', array=auc_values["sem"]),
        marker=dict(color="skyblue", line=dict(color="black", width=1)),
        showlegend=False
    ), row=1, col=2)

    # -------------------------
    # 3. Layout & Font
    # -------------------------

    fig.update_layout(
        height=400,
        width=1100,
        template="plotly_white",
        font=dict(size=16),
        legend=dict(title="Optimizers", font=dict(size=16)),
        margin=dict(t=60, l=50, r=50, b=50)
    )

    fig.update_xaxes(title_text="Normalized Trials", row=1, col=1)
    fig.update_yaxes(title_text="Normalized Inc-Performance", row=1, col=1)
    fig.update_xaxes(title_text="AUC", row=1, col=2)
    fig.update_yaxes(title_text="Optimizer", row=1, col=2)

    fig.show()


In [ ]:
# smac vs best fanova vs best hypershap
rnd_all[rnd_all['optimizer_id']=='random_adjust_cs_incumbent_cs_proba_hpi_thresh_0.5']

(5100, 20)

In [ ]:
perf_all[perf_all['optimizer_id']=='fanova_adjust_cs_incumbent_cs_proba_hpi_thresh_0.5']

(10200, 21)

In [ ]:
def get_commands_from_missing(missing):
    commands = []
    for opt_id, value in missing.items():
        for task_id, n_trials_norm, seed in value:
            # Construct the command dynamically, skipping None values
            command = f"python -m carps.experimenter.create_cluster_configs hydra.searchpath=[file:///home/daphne/Code/hpi_parego/package_hpi_parego/hpi_parego/configs] +optimizer/smac20=multiobjective_rf +customoptimizer=hpi_parego  'seed={seed}'"
            command += f" '+task/subselection/multiobjective/dev={task_id}'"
            baserundir = "results/"
            
            parts = opt_id.split('_')
            
            command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.hpi_method={parts[0]}"
            baserundir += f"{parts[0]}/"
            
            
            if 'adjust_cs_' in opt_id:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.adjust_cs=true"
                baserundir += f"adjust_cs_{parts[parts.index('cs')+1]}"
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.adjust_cs_method={parts[parts.index('cs')+1]}"
            else:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.adjust_cs=false"
            
            if 'constant' in opt_id:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.constant=true"
                baserundir += "_constant"
            else:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.constant=false"
            if 'cs_proba_hpi' in opt_id:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.cs_proba_hpi=true"
                baserundir += "_cs_proba_hpi"
            else:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.cs_proba_hpi=false"
            
            if 'adjust_prev_cfgs' in opt_id:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.adjust_previous_cfgs=true"
                baserundir += "_adjust_prev_cfgs"
            else:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.adjust_previous_cfgs=false"
            if 'set_to' in opt_id:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.set_to={parts[parts.index('to')+1]}"
                baserundir += f"_set_to_{parts[parts.index('to')+1]}"
            if 'dynamic_decay' in opt_id:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.dynamic_decay=linear"
                baserundir += "_dynamic_decay"
                if 'down' in opt_id:
                    command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.thresh_list=[0.9,0.8,0.7,0.6,0.5,0.4,0.3]"
                    baserundir += "_down"
                elif 'up' in opt_id:
                    command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.thresh_list=[0.3,0.4,0.5,0.6,0.7,0.8,0.9]"
                    baserundir += "_up"
            else:
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.dynamic_decay=none"
                command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.thresh={parts[parts.index('thresh')+1]}"
                baserundir += f"_thresh_{parts[parts.index('thresh')+1]}"
                

            command += f" baserundir={baserundir}"
            command += f" optimizer_id={parts[0]}_{baserundir.split('/')[-1]}"
            
            command += " -m"
            assert opt_id ==f"{parts[0]}_{baserundir.split('/')[-1]}"
            
            commands.append(command)
    return commands